Bayan Nezamabad - 20251971, Euan Bourke - 21332142

In [1]:
# Imports
import tensorflow as tf

In [ ]:
# Hyper Params
EPOCHS = 20
BATCH_SIZE = 128
VERBOSE = 1
OPTIMIZER = tf.keras.optimizers.Adam()
VALIDATION_SPLIT=0.90

IMG_ROW, IMG_COL = 224, 224 # Image dimensions
INPUT_SHAPE = (IMG_ROW, IMG_COL, 3) # 3 as RGB
NB_CLASSES = 100 # 100 different classifications of species

In [ ]:
class Dataset:
    def __init__(self):
        return

In [ ]:
class BENet: # Our implementation of AlexNet
    def __init__(self):
        return
